# Data Preprocessing for Lead Generation RAG Bot

This notebook preprocesses the WooCommerce product export CSV file to create optimized data for a **Lead Generation RAG Bot**. The bot will help convert website visitors into potential customers by providing relevant product information, pricing, and compelling descriptions.

## Lead Generation Objectives:
- Create customer-focused product descriptions
- Highlight key selling points and benefits
- Include pricing and availability information
- Generate compelling calls-to-action
- Optimize content for lead capture and conversion
- Handle customer inquiries about products and services

## Technical Objectives:
- Read and analyze the CSV file structure
- Clean and preprocess the data
- Handle missing values intelligently
- Create sales-oriented combined descriptions
- Export optimized data for RAG implementation

## 1. Import Required Libraries

Import all necessary libraries for data processing and cleaning.

In [6]:
import pandas as pd
import numpy as np
import re
import warnings
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Display options for better data viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Install Required Packages

Install packages for HTML content cleaning if not already installed.

In [7]:
# Install required packages for HTML cleaning
import subprocess
import sys


# Import the packages
from bs4 import BeautifulSoup
import html2text

print("All packages are ready!")

All packages are ready!


## 3. Load and Explore the Data

Load the CSV file and explore its structure.

In [3]:
# File path
input_file = "wc-product-export-29-6-2025-1751232970600.csv"

# Check if file exists
if not Path(input_file).exists():
    print(f"Error: File '{input_file}' not found!")
else:
    print(f"File '{input_file}' found. Loading data...")
    
    # Read the CSV file
    try:
        df = pd.read_csv(input_file, encoding='utf-8')
        print("File loaded with UTF-8 encoding")
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(input_file, encoding='latin1')
            print("File loaded with Latin-1 encoding")
        except Exception as e:
            print(f"Error loading file: {e}")
            df = None
    
    if df is not None:
        print(f"\nData loaded successfully!")
        print(f"Shape: {df.shape}")
        print(f"Columns: {len(df.columns)}")
        print(f"Rows: {len(df)}")

File 'wc-product-export-29-6-2025-1751232970600.csv' found. Loading data...
File loaded with UTF-8 encoding

Data loaded successfully!
Shape: (29, 86)
Columns: 86
Rows: 29


In [ ]:
df

In [8]:
# Display basic information about the dataset
if df is not None:
    print("=== DATASET OVERVIEW ===")
    print(f"Dataset shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\n=== COLUMN NAMES ===")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")
    
    print("\n=== DATA TYPES ===")
    print(df.dtypes.value_counts())
    
    print("\n=== MISSING VALUES ===")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing_data,
        'Missing Percentage': missing_percent
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    print(missing_df.head(10))

=== DATASET OVERVIEW ===
Dataset shape: (29, 86)
Memory usage: 0.10 MB

=== COLUMN NAMES ===
 1. ID
 2. Type
 3. SKU
 4. Name
 5. Published
 6. Is featured?
 7. Visibility in catalog
 8. Short description
 9. Description
10. Date sale price starts
11. Date sale price ends
12. Tax status
13. Tax class
14. In stock?
15. Stock
16. Low stock amount
17. Backorders allowed?
18. Sold individually?
19. Weight (kg)
20. Length (cm)
21. Width (cm)
22. Height (cm)
23. Allow customer reviews?
24. Purchase note
25. Sale price
26. Regular price
27. Categories
28. Tags
29. Shipping class
30. Images
31. Download limit
32. Download expiry days
33. Parent
34. Grouped products
35. Upsells
36. Cross-sells
37. External URL
38. Button text
39. Position
40. Meta: eltdf_show_new_sign_woo_meta
41. Meta: _yoast_wpseo_content_score
42. Meta: _yoast_wpseo_estimated-reading-time-minutes
43. Meta: _yoast_wpseo_wordproof_timestamp
44. Meta: rs_page_bg_color
45. Meta: _yoast_wpseo_primary_product_cat
46. Meta: _yoast_

In [9]:
# Display sample data from key columns
if df is not None:
    key_columns = ['ID', 'Type', 'Name', 'Categories', 'Regular price', 'Short description']
    existing_key_columns = [col for col in key_columns if col in df.columns]
    
    print("=== SAMPLE DATA ===")
    print("First 5 rows of key columns:")
    display(df[existing_key_columns].head())
    
    # Check product types
    if 'Type' in df.columns:
        print("\n=== PRODUCT TYPES ===")
        print(df['Type'].value_counts())
    
    # Check categories
    if 'Categories' in df.columns:
        print("\n=== TOP CATEGORIES ===")
        categories = df['Categories'].dropna().str.split(',').explode().str.strip()
        print(categories.value_counts().head(10))

=== SAMPLE DATA ===
First 5 rows of key columns:


,ID,Type,Name,Categories,Regular price,Short description
0,7595,simple,Ginger Cocktails,Buffet,36.0,"Lorem ipsum dolor sit amet, consectetur adipisicing elitsi edo eiusmod senteas tempor incididunt..."
1,7634,simple,Buffet de soutenance Chic standard,"Buffet, Buffet > Buffet de soutenance",6000.0,Concentrez vous sur votre oral et faites confiance à HS traiteur pour le buffet de votre soutena...
2,7636,simple,Décoration et Matériel Chic,Buffet,1000.0,"<span class=""JsGRdQ"">Le buffet chic apporte une touche d'élégance et de gourmandise à votre éven..."
3,7637,simple,Pastilla duo,Cuisine marocaine et internationale,1500.0,C'est une composition de deux pastillas. La première est une pastilla aux saveurs océans : poiss...
4,7700,simple,Chewa ou Mechwi Royal,"Cuisine marocaine et internationale, Mariage et fiançialle",1300.0,"Epaules d'agneau marinées et cuites à la façon royale, accompagnées de bouquetière de légumes ou..."



=== PRODUCT TYPES ===
Type
simple       13
variation    11
variable      5
Name: count, dtype: int64

=== TOP CATEGORIES ===
Categories
Buffet                                 9
Cuisine marocaine et internationale    7
Mariage et fiançialle                  5
Buffet > Buffet de soutenance          4
Pâtisserie                             3
Salade                                 1
Anniversaire\                          1
naissance et baptème                   1
Name: count, dtype: int64


## 4. Define Cleaning Functions

Create functions to clean HTML content and handle text preprocessing.

In [10]:
def clean_html_content(text):
    """
    Clean HTML content and convert it to plain text
    """
    if pd.isna(text) or text == '':
        return ''
    
    # Convert HTML to text
    h = html2text.HTML2Text()
    h.ignore_links = False
    h.body_width = 0
    
    try:
        # Use BeautifulSoup to handle malformed HTML
        soup = BeautifulSoup(str(text), 'html.parser')
        cleaned_text = soup.get_text()
        
        # Further clean with html2text
        cleaned_text = h.handle(cleaned_text)
        
        # Remove extra whitespaces and newlines
        cleaned_text = re.sub(r'\n+', ' ', cleaned_text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        
        # Remove special characters but keep basic punctuation
        cleaned_text = re.sub(r'[^\w\s\.,!?;:-]', '', cleaned_text)
        
        return cleaned_text.strip()
    except Exception as e:
        print(f"Error cleaning text: {e}")
        return str(text).strip()

def clean_text_field(text):
    """
    Clean general text fields
    """
    if pd.isna(text) or text == '':
        return ''
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', str(text))
    
    # Remove leading/trailing whitespace
    return text.strip()

# Test the cleaning functions
sample_html = "<p>This is a <strong>sample</strong> HTML content with <em>formatting</em>.</p>"
cleaned_sample = clean_html_content(sample_html)
print(f"Original: {sample_html}")
print(f"Cleaned: {cleaned_sample}")

Original: <p>This is a <strong>sample</strong> HTML content with <em>formatting</em>.</p>
Cleaned: This is a sample HTML content with formatting.


## 5. Select Relevant Columns

Identify and select columns that are relevant for RAG purposes.

In [11]:
# Define relevant columns for Lead Generation RAG
relevant_columns = [
    'ID', 'Type', 'SKU', 'Name', 'Published', 'Short description', 
    'Description', 'In stock?', 'Stock', 'Regular price', 'Sale price',
    'Categories', 'Tags', 'Images', 'Purchase note', 'Button text'
]

# Check which columns exist in our dataset
if df is not None:
    existing_columns = [col for col in relevant_columns if col in df.columns]
    missing_columns = [col for col in relevant_columns if col not in df.columns]
    
    print("=== COLUMN ANALYSIS FOR LEAD GENERATION ===")
    print(f"Columns we want: {len(relevant_columns)}")
    print(f"Columns available: {len(existing_columns)}")
    print(f"Columns missing: {len(missing_columns)}")
    
    print("\n=== AVAILABLE COLUMNS (Lead Gen Priority) ===")
    # Prioritize columns by lead generation importance
    lead_priority = {
        'Name': 'HIGH - Product identification',
        'Short description': 'HIGH - Quick pitch',
        'Description': 'HIGH - Detailed selling points',
        'Regular price': 'HIGH - Pricing info',
        'Sale price': 'HIGH - Special offers',
        'Categories': 'MEDIUM - Product classification',
        'Tags': 'MEDIUM - Keywords for targeting',
        'In stock?': 'MEDIUM - Availability',
        'Images': 'MEDIUM - Visual appeal',
        'Purchase note': 'MEDIUM - Additional sales info',
        'Button text': 'LOW - CTA text',
        'Stock': 'LOW - Inventory details',
        'Type': 'LOW - Technical classification',
        'SKU': 'LOW - Internal reference',
        'ID': 'LOW - Database reference'
    }
    
    for col in existing_columns:
        non_null_count = df[col].notna().sum()
        priority = lead_priority.get(col, 'LOW - Other')
        print(f"✓ {col} ({non_null_count}/{len(df)} non-null) - {priority}")
    
    if missing_columns:
        print("\n=== MISSING COLUMNS ===")
        for col in missing_columns:
            priority = lead_priority.get(col, 'LOW - Other')
            print(f"✗ {col} - {priority}")
    
    # Create filtered dataframe
    df_filtered = df[existing_columns].copy()
    print(f"\nFiltered dataset shape: {df_filtered.shape}")
    print("✅ Data filtered for lead generation optimization")

=== COLUMN ANALYSIS FOR LEAD GENERATION ===
Columns we want: 16
Columns available: 16
Columns missing: 0

=== AVAILABLE COLUMNS (Lead Gen Priority) ===
✓ ID (29/29 non-null) - LOW - Database reference
✓ Type (29/29 non-null) - LOW - Technical classification
✓ SKU (4/29 non-null) - LOW - Internal reference
✓ Name (29/29 non-null) - HIGH - Product identification
✓ Published (29/29 non-null) - LOW - Other
✓ Short description (16/29 non-null) - HIGH - Quick pitch
✓ Description (19/29 non-null) - HIGH - Detailed selling points
✓ In stock? (29/29 non-null) - MEDIUM - Availability
✓ Stock (0/29 non-null) - LOW - Inventory details
✓ Regular price (23/29 non-null) - HIGH - Pricing info
✓ Sale price (4/29 non-null) - HIGH - Special offers
✓ Categories (18/29 non-null) - MEDIUM - Product classification
✓ Tags (13/29 non-null) - MEDIUM - Keywords for targeting
✓ Images (16/29 non-null) - MEDIUM - Visual appeal
✓ Purchase note (0/29 non-null) - MEDIUM - Additional sales info
✓ Button text (0/29 non

## 6. Data Cleaning and Preprocessing

Clean the data by handling missing values, filtering invalid records, and preprocessing text fields.

In [12]:
# Start with a copy of the filtered data
df_clean = df_filtered.copy()

print("=== INITIAL CLEANING ===")
print(f"Starting with {len(df_clean)} rows")

# 1. Filter only published products
if 'Published' in df_clean.columns:
    df_clean = df_clean[df_clean['Published'] == 1].copy()
    print(f"After filtering published products: {len(df_clean)} rows")

# 2. Remove rows without product names
if 'Name' in df_clean.columns:
    df_clean = df_clean.dropna(subset=['Name']).copy()
    df_clean = df_clean[df_clean['Name'].str.strip() != ''].copy()
    print(f"After removing products without names: {len(df_clean)} rows")

# 3. Clean text fields
print("\n=== CLEANING TEXT FIELDS ===")
text_fields = ['Name', 'Short description', 'Description', 'Categories', 'Tags', 'SKU']

for field in text_fields:
    if field in df_clean.columns:
        print(f"Cleaning {field}...")
        if field in ['Short description', 'Description']:
            # Clean HTML content
            df_clean[field] = df_clean[field].apply(clean_html_content)
        else:
            # Clean regular text
            df_clean[field] = df_clean[field].apply(clean_text_field)

print("Text fields cleaned successfully!")

=== INITIAL CLEANING ===
Starting with 29 rows
After filtering published products: 0 rows
After removing products without names: 0 rows

=== CLEANING TEXT FIELDS ===
Cleaning Name...
Cleaning Short description...
Cleaning Description...
Cleaning Categories...
Cleaning Tags...
Cleaning SKU...
Text fields cleaned successfully!


In [13]:
# 4. Handle missing values
print("=== HANDLING MISSING VALUES ===")

# Fill missing text fields with empty strings
text_columns = ['Short description', 'Description', 'Categories', 'Tags', 'SKU']
for col in text_columns:
    if col in df_clean.columns:
        missing_count = df_clean[col].isna().sum()
        df_clean[col] = df_clean[col].fillna('')
        print(f"{col}: Filled {missing_count} missing values with empty string")

# Handle stock information
if 'In stock?' in df_clean.columns:
    missing_count = df_clean['In stock?'].isna().sum()
    df_clean['In stock?'] = df_clean['In stock?'].fillna(0)
    print(f"In stock?: Filled {missing_count} missing values with 0")

if 'Stock' in df_clean.columns:
    missing_count = df_clean['Stock'].isna().sum()
    df_clean['Stock'] = pd.to_numeric(df_clean['Stock'], errors='coerce').fillna(0)
    print(f"Stock: Filled {missing_count} missing values with 0")

# Handle price information
price_columns = ['Regular price', 'Sale price']
for col in price_columns:
    if col in df_clean.columns:
        missing_count = df_clean[col].isna().sum()
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0)
        print(f"{col}: Converted to numeric and filled {missing_count} missing values with 0")

print(f"\nCleaned dataset shape: {df_clean.shape}")

=== HANDLING MISSING VALUES ===
Short description: Filled 0 missing values with empty string
Description: Filled 0 missing values with empty string
Categories: Filled 0 missing values with empty string
Tags: Filled 0 missing values with empty string
SKU: Filled 0 missing values with empty string
In stock?: Filled 0 missing values with 0
Stock: Filled 0 missing values with 0
Regular price: Converted to numeric and filled 0 missing values with 0
Sale price: Converted to numeric and filled 0 missing values with 0

Cleaned dataset shape: (0, 16)


## 7. Create Combined Descriptions for RAG

Combine multiple product information fields into a comprehensive description suitable for RAG.

In [15]:
def create_lead_generation_description(row):
    """
    Create a compelling product description optimized for lead generation
    Focus on benefits, urgency, and conversion elements
    """
    description_parts = []
    
    # Product name with appeal
    if pd.notna(row.get('Name')) and row.get('Name', '').strip() != '':
        description_parts.append(f"🍽️ {row['Name']}")
    
    # Product type and category for targeting
    if pd.notna(row.get('Type')) and row.get('Type', '').strip() != '':
        description_parts.append(f"Service Type: {row['Type']}")
    
    if pd.notna(row.get('Categories')) and row.get('Categories', '').strip() != '':
        categories = row['Categories'].replace(',', ' • ')
        description_parts.append(f"Categories: {categories}")
    
    # Price information with sales appeal
    regular_price = row.get('Regular price', 0)
    sale_price = row.get('Sale price', 0)
    
    if regular_price > 0:
        if sale_price > 0 and sale_price < regular_price:
            savings = regular_price - sale_price
            savings_percent = (savings / regular_price) * 100
            description_parts.append(f"💰 SPECIAL OFFER: {sale_price} MAD (Save {savings} MAD - {savings_percent:.0f}% OFF! Regular price: {regular_price} MAD)")
        else:
            description_parts.append(f"💰 Price: {regular_price} MAD")
    
    # Short description as elevator pitch
    if pd.notna(row.get('Short description')) and row.get('Short description', '').strip() != '':
        pitch = row['Short description']
        # Add appeal if it doesn't already have it
        if not any(word in pitch.lower() for word in ['perfect', 'ideal', 'amazing', 'delicious', 'special', 'unique']):
            pitch = f"Perfect for your needs: {pitch}"
        description_parts.append(f"🎯 Why Choose This: {pitch}")
    
    # Full description with benefits focus
    if pd.notna(row.get('Description')) and row.get('Description', '').strip() != '':
        description = row['Description']
        # Truncate very long descriptions for better readability
        if len(description) > 300:
            description = description[:300] + "... [Contact us for full details]"
        description_parts.append(f"📋 Full Details: {description}")
    
    # Availability with urgency
    stock_status = row.get('In stock?', 0)
    if stock_status == 1:
        stock_qty = row.get('Stock', 0)
        if stock_qty > 0:
            if stock_qty <= 5:
                description_parts.append(f"⚡ LIMITED AVAILABILITY: Only {stock_qty} units remaining - Book now!")
            else:
                description_parts.append(f"✅ Available Now: {stock_qty} units in stock")
        else:
            description_parts.append("✅ Available Now - Ready to order!")
    else:
        description_parts.append("📞 Currently unavailable - Contact us for alternatives or pre-booking!")
    
    # Tags as selling points
    if pd.notna(row.get('Tags')) and row.get('Tags', '').strip() != '':
        tags = row['Tags'].replace(',', ' • ')
        description_parts.append(f"🏷️ Highlights: {tags}")
    
    # Purchase notes as additional selling points
    if pd.notna(row.get('Purchase note')) and row.get('Purchase note', '').strip() != '':
        description_parts.append(f"💡 Important: {row['Purchase note']}")
    
    # SKU for reference
    if pd.notna(row.get('SKU')) and row.get('SKU', '').strip() != '':
        description_parts.append(f"🔖 Reference: {row['SKU']}")
    
    # Call to action
    cta_text = row.get('Button text', '').strip() if pd.notna(row.get('Button text')) else ''
    if not cta_text:
        if stock_status == 1:
            cta_text = "Order Now - Contact HS Traiteur Today!"
        else:
            cta_text = "Get Quote - Contact HS Traiteur for Details!"
    
    description_parts.append(f"🚀 Next Step: {cta_text}")
    
    return ' | '.join(description_parts)

def add_lead_generation_metadata(row):
    """
    Add metadata useful for lead generation targeting
    """
    metadata = {}
    
    # Price tier for targeting
    price = row.get('Regular price', 0)
    if price > 0:
        if price <= 500:
            metadata['price_tier'] = 'budget_friendly'
        elif price <= 2000:
            metadata['price_tier'] = 'mid_range'
        else:
            metadata['price_tier'] = 'premium'
    else:
        metadata['price_tier'] = 'contact_for_pricing'
    
    # Availability urgency
    stock_status = row.get('In stock?', 0)
    stock_qty = row.get('Stock', 0)
    
    if stock_status == 1:
        if stock_qty <= 5 and stock_qty > 0:
            metadata['urgency'] = 'high'
        elif stock_qty > 5:
            metadata['urgency'] = 'low'
        else:
            metadata['urgency'] = 'medium'
    else:
        metadata['urgency'] = 'contact_required'
    
    # Has special offer
    regular_price = row.get('Regular price', 0)
    sale_price = row.get('Sale price', 0)
    metadata['has_special_offer'] = sale_price > 0 and sale_price < regular_price
    
    return metadata

# Apply the functions to create lead generation optimized descriptions
print("Creating lead generation optimized descriptions...")
df_clean = df_filtered.copy()  # Use df_filtered instead of df_clean which is empty
df_clean['lead_gen_description'] = df_clean.apply(create_lead_generation_description, axis=1)
df_clean['lead_gen_metadata'] = df_clean.apply(add_lead_generation_metadata, axis=1)

# Remove products with empty descriptions
df_clean = df_clean[df_clean['lead_gen_description'] != ''].copy()

print(f"Products with lead generation descriptions: {len(df_clean)}")
# Check if the dataframe is not empty before calculating mean
if len(df_clean) > 0:
    print(f"Average description length: {df_clean['lead_gen_description'].str.len().mean():.1f} characters")
else:
    print("No products with lead generation descriptions found.")

# Analyze lead generation potential
print("\n=== LEAD GENERATION ANALYSIS ===")
price_tiers = [meta.get('price_tier', 'unknown') for meta in df_clean['lead_gen_metadata']]
urgency_levels = [meta.get('urgency', 'unknown') for meta in df_clean['lead_gen_metadata']]
special_offers = [meta.get('has_special_offer', False) for meta in df_clean['lead_gen_metadata']]

print(f"Price distribution: {pd.Series(price_tiers).value_counts().to_dict()}")
print(f"Urgency distribution: {pd.Series(urgency_levels).value_counts().to_dict()}")
print(f"Products with special offers: {sum(special_offers)}/{len(special_offers)} ({sum(special_offers)/len(special_offers)*100:.1f}%)")

Creating lead generation optimized descriptions...
Products with lead generation descriptions: 29
Average description length: 632.6 characters

=== LEAD GENERATION ANALYSIS ===
Price distribution: {'premium': 11, 'mid_range': 8, 'contact_for_pricing': 6, 'budget_friendly': 4}
Urgency distribution: {'medium': 27, 'contact_required': 2}
Products with special offers: 4/29 (13.8%)


In [16]:
# Display sample lead generation descriptions
print("=== SAMPLE LEAD GENERATION DESCRIPTIONS ===")
for i, (idx, row) in enumerate(df_clean.head(3).iterrows()):
    print(f"\n{'='*70}")
    print(f"LEAD GENERATION SAMPLE {i+1}")
    print(f"{'='*70}")
    print(f"Product: {row.get('Name', 'N/A')}")
    
    # Show lead gen metadata
    metadata = row.get('lead_gen_metadata', {})
    print(f"Price Tier: {metadata.get('price_tier', 'N/A')}")
    print(f"Urgency Level: {metadata.get('urgency', 'N/A')}")
    print(f"Special Offer: {'Yes' if metadata.get('has_special_offer', False) else 'No'}")
    
    print(f"\n📝 Lead Generation Description:")
    desc = row['lead_gen_description']
    # Split into components for better readability
    parts = desc.split(' | ')
    for j, part in enumerate(parts, 1):
        print(f"  {j}. {part}")
    
    print(f"\n💭 Bot Usage Example:")
    print(f"   Customer: 'Tell me about catering options for graduation party'")
    print(f"   Bot: 'I found a perfect match! {parts[0] if parts else row.get('Name', 'Product')}...'")
    print("-" * 70)

=== SAMPLE LEAD GENERATION DESCRIPTIONS ===

LEAD GENERATION SAMPLE 1
Product: Ginger Cocktails
Price Tier: budget_friendly
Urgency Level: medium
Special Offer: No

📝 Lead Generation Description:
  1. 🍽️ Ginger Cocktails
  2. Service Type: simple
  3. Categories: Buffet
  4. 💰 Price: 36.0 MAD
  5. 🎯 Why Choose This: Perfect for your needs: Lorem ipsum dolor sit amet, consectetur adipisicing elitsi edo eiusmod senteas tempor incididunt ut labore et dolr emagna aliqua. Ut enim ad minim dano ris veniam quis
  6. 📋 Full Details: Lorem ipsum dolor sit amet, consectetur adipisicing elitsi edo eiusmod senteas tempor incididunt ut labore et dolr emagna aliqua. Ut enim ad minim dano ris veniam quis
  7. ✅ Available Now - Ready to order!
  8. 🚀 Next Step: Order Now - Contact HS Traiteur Today!

💭 Bot Usage Example:
   Customer: 'Tell me about catering options for graduation party'
   Bot: 'I found a perfect match! 🍽️ Ginger Cocktails...'
----------------------------------------------------------

## 8. Create Final Dataset for RAG

Select the most relevant columns and create the final clean dataset.

In [17]:
# Define final columns for Lead Generation RAG
rag_columns = [
    'ID', 'Name', 'Type', 'SKU', 'Categories', 'Tags',
    'Regular price', 'Sale price', 'In stock?', 'Stock',
    'lead_gen_description', 'lead_gen_metadata'
]

# Select only existing columns
final_columns = [col for col in rag_columns if col in df_clean.columns]
df_rag = df_clean[final_columns].copy()

# Extract metadata into separate columns for easier analysis
df_rag['price_tier'] = df_rag['lead_gen_metadata'].apply(lambda x: x.get('price_tier', 'unknown') if isinstance(x, dict) else 'unknown')
df_rag['urgency_level'] = df_rag['lead_gen_metadata'].apply(lambda x: x.get('urgency', 'unknown') if isinstance(x, dict) else 'unknown')
df_rag['has_special_offer'] = df_rag['lead_gen_metadata'].apply(lambda x: x.get('has_special_offer', False) if isinstance(x, dict) else False)

print("=== FINAL LEAD GENERATION RAG DATASET ===")
print(f"Shape: {df_rag.shape}")
print(f"Columns: {final_columns}")

# Display lead generation specific statistics
print("\n=== LEAD GENERATION STATISTICS ===")

if 'Type' in df_rag.columns:
    print("\nProduct Types (Lead Targeting):")
    type_counts = df_rag['Type'].value_counts()
    for ptype, count in type_counts.items():
        percentage = (count / len(df_rag)) * 100
        print(f"  {ptype}: {count} products ({percentage:.1f}%)")

if 'Categories' in df_rag.columns:
    print("\nTop Categories (Lead Segmentation):")
    categories = df_rag['Categories'].str.split(',').explode().str.strip()
    categories = categories[categories != '']
    top_categories = categories.value_counts().head(10)
    for cat, count in top_categories.items():
        print(f"  {cat}: {count} products")

# Price analysis for lead qualification
if 'Regular price' in df_rag.columns:
    prices = df_rag['Regular price'][df_rag['Regular price'] > 0]
    if len(prices) > 0:
        print(f"\nPricing Analysis (Lead Qualification):")
        print(f"  Price range: {prices.min()} - {prices.max()} MAD")
        print(f"  Average order value: {prices.mean():.2f} MAD")
        print(f"  Median price: {prices.median():.2f} MAD")
        
        # Price tier distribution
        print(f"\nPrice Tier Distribution:")
        tier_counts = df_rag['price_tier'].value_counts()
        for tier, count in tier_counts.items():
            percentage = (count / len(df_rag)) * 100
            print(f"  {tier.replace('_', ' ').title()}: {count} products ({percentage:.1f}%)")

# Availability and urgency analysis
print(f"\nAvailability & Urgency Analysis:")
if 'In stock?' in df_rag.columns:
    in_stock_count = df_rag['In stock?'].sum()
    total_products = len(df_rag)
    print(f"  Products available: {in_stock_count}/{total_products} ({in_stock_count/total_products*100:.1f}%)")

urgency_counts = df_rag['urgency_level'].value_counts()
for urgency, count in urgency_counts.items():
    percentage = (count / len(df_rag)) * 100
    print(f"  {urgency.replace('_', ' ').title()} urgency: {count} products ({percentage:.1f}%)")

# Special offers analysis
special_offer_count = df_rag['has_special_offer'].sum()
print(f"\nSpecial Offers (Conversion Boosters):")
print(f"  Products with special offers: {special_offer_count}/{len(df_rag)} ({special_offer_count/len(df_rag)*100:.1f}%)")

# Description analysis for RAG optimization
print(f"\nDescription Analysis (RAG Optimization):")
desc_lengths = df_rag['lead_gen_description'].str.len()
print(f"  Min length: {desc_lengths.min()} characters")
print(f"  Max length: {desc_lengths.max()} characters")
print(f"  Average length: {desc_lengths.mean():.1f} characters")
print(f"  Optimal for RAG: {((desc_lengths >= 100) & (desc_lengths <= 800)).sum()} products")

print("\n✅ Dataset optimized for lead generation RAG bot!")

=== FINAL LEAD GENERATION RAG DATASET ===
Shape: (29, 15)
Columns: ['ID', 'Name', 'Type', 'SKU', 'Categories', 'Tags', 'Regular price', 'Sale price', 'In stock?', 'Stock', 'lead_gen_description', 'lead_gen_metadata']

=== LEAD GENERATION STATISTICS ===

Product Types (Lead Targeting):
  simple: 13 products (44.8%)
  variation: 11 products (37.9%)
  variable: 5 products (17.2%)

Top Categories (Lead Segmentation):
  Buffet: 9 products
  Cuisine marocaine et internationale: 7 products
  Mariage et fiançialle: 5 products
  Buffet > Buffet de soutenance: 4 products
  Pâtisserie: 3 products
  Salade: 1 products
  Anniversaire\: 1 products
  naissance et baptème: 1 products

Pricing Analysis (Lead Qualification):
  Price range: 36.0 - 13100.0 MAD
  Average order value: 3800.26 MAD
  Median price: 1500.00 MAD

Price Tier Distribution:
  Premium: 11 products (37.9%)
  Mid Range: 8 products (27.6%)
  Contact For Pricing: 6 products (20.7%)
  Budget Friendly: 4 products (13.8%)

Availability & U

In [18]:
# Display the final dataset structure
print("=== FINAL DATASET PREVIEW ===")
display(df_rag.head())

print("\n=== DATA TYPES ===")
print(df_rag.dtypes)

print("\n=== MISSING VALUES IN FINAL DATASET ===")
missing_final = df_rag.isnull().sum()
if missing_final.sum() > 0:
    print(missing_final[missing_final > 0])
else:
    print("No missing values in the final dataset!")

=== FINAL DATASET PREVIEW ===


,ID,Name,Type,SKU,Categories,Tags,Regular price,Sale price,In stock?,Stock,lead_gen_description,lead_gen_metadata,price_tier,urgency_level,has_special_offer
0,7595,Ginger Cocktails,simple,NaN,Buffet,NaN,36.0,NaN,1,NaN,🍽️ Ginger Cocktails | Service Type: simple | Categories: Buffet | 💰 Price: 36.0 MAD | 🎯 Why Choo...,"{'price_tier': 'budget_friendly', 'urgency': 'medium', 'has_special_offer': False}",budget_friendly,medium,False
1,7634,Buffet de soutenance Chic standard,simple,NaN,"Buffet, Buffet > Buffet de soutenance","buffet, doctorat, école, étudiant, soutenance, thèse",6000.0,NaN,1,NaN,🍽️ Buffet de soutenance Chic standard | Service Type: simple | Categories: Buffet • Buffet > Bu...,"{'price_tier': 'premium', 'urgency': 'medium', 'has_special_offer': False}",premium,medium,False
2,7636,Décoration et Matériel Chic,simple,NaN,Buffet,NaN,1000.0,950.0,1,NaN,🍽️ Décoration et Matériel Chic | Service Type: simple | Categories: Buffet | 💰 SPECIAL OFFER: 95...,"{'price_tier': 'mid_range', 'urgency': 'medium', 'has_special_offer': True}",mid_range,medium,True
3,7637,Pastilla duo,simple,NaN,Cuisine marocaine et internationale,"Pastilla, poisson, poulet",1500.0,1300.0,1,NaN,🍽️ Pastilla duo | Service Type: simple | Categories: Cuisine marocaine et internationale | 💰 SPE...,"{'price_tier': 'mid_range', 'urgency': 'medium', 'has_special_offer': True}",mid_range,medium,True
4,7700,Chewa ou Mechwi Royal,simple,NaN,"Cuisine marocaine et internationale, Mariage et fiançialle",Viande,1300.0,NaN,1,NaN,🍽️ Chewa ou Mechwi Royal | Service Type: simple | Categories: Cuisine marocaine et international...,"{'price_tier': 'mid_range', 'urgency': 'medium', 'has_special_offer': False}",mid_range,medium,False



=== DATA TYPES ===
ID                        int64
Name                     object
Type                     object
SKU                      object
Categories               object
Tags                     object
Regular price           float64
Sale price              float64
In stock?                 int64
Stock                   float64
lead_gen_description     object
lead_gen_metadata        object
price_tier               object
urgency_level            object
has_special_offer          bool
dtype: object

=== MISSING VALUES IN FINAL DATASET ===
SKU              25
Categories       11
Tags             16
Regular price     6
Sale price       25
Stock            29
dtype: int64


## 9. Save the Clean Dataset

Export the cleaned and processed data to a new CSV file.

In [19]:
# Define output filenames for lead generation
output_file = "lead_generation_products_rag.csv"
metadata_file = "lead_generation_metadata.csv"

# Prepare the final dataset for RAG
df_rag_final = df_rag.copy()

# Create a separate metadata file for advanced targeting
metadata_df = df_rag[['ID', 'Name', 'price_tier', 'urgency_level', 'has_special_offer', 'Categories', 'Tags']].copy()

# Save the main lead generation dataset
try:
    df_rag_final.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Lead Generation RAG dataset saved to: {output_file}")
    print(f"📊 Dataset contains {len(df_rag_final)} products optimized for lead generation")
    
    # File size information
    file_size = Path(output_file).stat().st_size / 1024  # KB
    print(f"📁 Main file size: {file_size:.1f} KB")
    
    # Save metadata file for targeting
    metadata_df.to_csv(metadata_file, index=False, encoding='utf-8')
    metadata_size = Path(metadata_file).stat().st_size / 1024  # KB
    print(f"📁 Metadata file saved: {metadata_file} ({metadata_size:.1f} KB)")
    
    print(f"\n🎯 LEAD GENERATION READY:")
    print(f"   • Main RAG file: {output_file}")
    print(f"   • Targeting metadata: {metadata_file}")
    print(f"   • Total products: {len(df_rag_final)}")
    print(f"   • Products with special offers: {df_rag_final['has_special_offer'].sum()}")
    print(f"   • High urgency products: {(df_rag_final['urgency_level'] == 'high').sum()}")
    
except Exception as e:
    print(f"❌ Error saving files: {e}")

✅ Lead Generation RAG dataset saved to: lead_generation_products_rag.csv
📊 Dataset contains 29 products optimized for lead generation
📁 Main file size: 24.9 KB
📁 Metadata file saved: lead_generation_metadata.csv (3.0 KB)

🎯 LEAD GENERATION READY:
   • Main RAG file: lead_generation_products_rag.csv
   • Targeting metadata: lead_generation_metadata.csv
   • Total products: 29
   • Products with special offers: 4
   • High urgency products: 0


## 10. Quality Check and Validation

Perform final quality checks on the cleaned dataset.

In [ ]:
# Quality checks
print("=== QUALITY CHECKS ===")

# Check 1: All products have names
products_without_names = df_rag['Name'].isna().sum() if 'Name' in df_rag.columns else 0
print(f"✅ Products without names: {products_without_names}")

# Check 2: All products have combined descriptions
empty_descriptions = (df_rag['combined_description'] == '').sum()
print(f"✅ Products with empty descriptions: {empty_descriptions}")

# Check 3: Price validation
if 'Regular price' in df_rag.columns:
    negative_prices = (df_rag['Regular price'] < 0).sum()
    print(f"✅ Products with negative prices: {negative_prices}")

# Check 4: Description length distribution
desc_lengths = df_rag['combined_description'].str.len()
very_short = (desc_lengths < 50).sum()
very_long = (desc_lengths > 1000).sum()
print(f"✅ Very short descriptions (<50 chars): {very_short}")
print(f"✅ Very long descriptions (>1000 chars): {very_long}")

# Check 5: Duplicate products
if 'Name' in df_rag.columns:
    duplicates = df_rag['Name'].duplicated().sum()
    print(f"✅ Duplicate product names: {duplicates}")

print("\n=== FINAL SUMMARY ===")
print(f"🎯 Original dataset: {df.shape[0] if df is not None else 0} products")
print(f"🎯 Cleaned dataset: {len(df_rag)} products")
print(f"🎯 Data reduction: {((df.shape[0] - len(df_rag)) / df.shape[0] * 100):.1f}%" if df is not None else "N/A")
print(f"🎯 Columns reduced from {len(df.columns) if df is not None else 0} to {len(df_rag.columns)}")
print(f"✅ Dataset is ready for RAG implementation!")

## 11. Sample Data for RAG Testing

Display some sample products that are ready for RAG testing.

In [ ]:
# Display sample products for lead generation RAG testing
print("=== LEAD GENERATION BOT CONVERSATION EXAMPLES ===")

# Select a diverse sample for different scenarios
sample_size = min(3, len(df_rag))
sample_df = df_rag.sample(n=sample_size, random_state=42) if len(df_rag) > sample_size else df_rag

lead_scenarios = [
    "Customer asks about wedding catering options",
    "Customer looking for graduation party buffet", 
    "Customer inquiring about corporate event catering"
]

for i, (idx, row) in enumerate(sample_df.iterrows(), 1):
    print(f"\n{'='*80}")
    print(f"LEAD GENERATION SCENARIO {i}: {lead_scenarios[min(i-1, len(lead_scenarios)-1)]}")
    print(f"{'='*80}")
    
    # Product information
    print(f"🍽️ Product: {row.get('Name', 'N/A')}")
    print(f"🏷️  Category: {row.get('Categories', 'N/A')}")
    print(f"💰 Price: {row.get('Regular price', 'N/A')} MAD")
    print(f"📊 Lead Tier: {row.get('price_tier', 'N/A').replace('_', ' ').title()}")
    print(f"⚡ Urgency: {row.get('urgency_level', 'N/A').replace('_', ' ').title()}")
    print(f"🎁 Special Offer: {'Yes' if row.get('has_special_offer', False) else 'No'}")
    
    # Bot conversation example
    print(f"\n🤖 BOT CONVERSATION EXAMPLE:")
    print(f"📱 Customer: \"Hi, I'm looking for catering for my special event\"")
    print(f"🤖 Bot: \"Great! I'd love to help you find the perfect catering solution.\"")
    print(f"")
    
    # Extract key selling points from description
    desc_parts = row['lead_gen_description'].split(' | ')
    product_name = desc_parts[0] if desc_parts else row.get('Name', 'This product')
    
    print(f"🤖 Bot: \"{product_name} might be perfect for you!\"")
    
    # Show pricing appeal
    if row.get('has_special_offer', False):
        print(f"🤖 Bot: \"I have some great news - we currently have a special offer on this item!\"")
    
    # Show urgency if applicable
    if row.get('urgency_level') == 'high':
        print(f"🤖 Bot: \"Just so you know, this is quite popular and we have limited availability right now.\"")
    
    print(f"🤖 Bot: \"Would you like me to provide more details or help you get a custom quote?\"")
    print(f"📱 Customer: \"Yes, please tell me more!\"")
    print(f"🤖 Bot: \"Perfect! Let me share the details...\"")
    
    # Show RAG retrieved information
    print(f"\n📋 RAG RETRIEVED INFORMATION:")
    rag_parts = row['lead_gen_description'].split(' | ')
    for j, part in enumerate(rag_parts[:4], 1):  # Show first 4 parts
        print(f"   {j}. {part}")
    if len(rag_parts) > 4:
        print(f"   ... and {len(rag_parts)-4} more details")
    
    print(f"\n🎯 LEAD CAPTURE OPPORTUNITY:")
    print(f"🤖 Bot: \"To provide you with an accurate quote and ensure availability,\"")
    print(f"     \"could I get your contact information and event details?\"")
    print(f"📝 Lead Form: Name, Phone, Email, Event Date, Guest Count")
    print(f"")

print(f"\n{'='*80}")
print("🚀 LEAD GENERATION RAG BOT OPTIMIZATION COMPLETE!")
print(f"{'='*80}")

print(f"\n📊 FINAL LEAD GENERATION SUMMARY:")
print(f"   📁 Main dataset: {output_file}")
print(f"   📁 Metadata file: {metadata_file}")
print(f"   🎯 Total products: {len(df_rag)} (optimized for conversion)")
print(f"   💰 Price tiers: {df_rag['price_tier'].nunique()} different targeting levels")
print(f"   ⚡ Urgency levels: {df_rag['urgency_level'].nunique()} different urgency types")
print(f"   🎁 Special offers: {df_rag['has_special_offer'].sum()} products with promotions")
print(f"   📝 Avg description: {df_rag['lead_gen_description'].str.len().mean():.0f} chars (RAG optimized)")

print(f"\n✅ Your lead generation RAG bot is ready to:")
print(f"   • Convert visitors into qualified leads")
print(f"   • Provide compelling product information") 
print(f"   • Create urgency and encourage action")
print(f"   • Capture contact information naturally")
print(f"   • Segment leads by price tier and urgency")

print(f"\n🎯 Next steps:")
print(f"   1. Implement RAG system with the generated descriptions")
print(f"   2. Set up lead capture forms integrated with bot responses")
print(f"   3. Configure different conversation flows based on price tiers")
print(f"   4. Monitor conversion rates and optimize descriptions")
print(f"   5. A/B test different urgency messaging")

## 12. Lead Generation Bot Configuration Recommendations

Based on the data analysis, here are specific recommendations for implementing your lead generation RAG bot:

In [ ]:
# Lead Generation Bot Configuration Analysis
print("=== LEAD GENERATION BOT CONFIGURATION RECOMMENDATIONS ===")

# Analyze the data for bot configuration
total_products = len(df_rag)
high_urgency = (df_rag['urgency_level'] == 'high').sum()
special_offers = df_rag['has_special_offer'].sum()
premium_products = (df_rag['price_tier'] == 'premium').sum()
budget_products = (df_rag['price_tier'] == 'budget_friendly').sum()

print(f"\n1. 🎯 CONVERSATION FLOW RECOMMENDATIONS:")
print(f"   • Prioritize {high_urgency} high-urgency products for immediate response")
print(f"   • Use special offer messaging for {special_offers} products with promotions")
print(f"   • Create premium consultation flow for {premium_products} high-value items")
print(f"   • Offer quick booking for {budget_products} budget-friendly options")

print(f"\n2. 🔥 URGENCY TRIGGERS:")
urgency_triggers = {
    'high': 'Limited availability - Book now before it\'s gone!',
    'medium': 'Popular choice - Recommended to reserve early',
    'low': 'Available for booking - Flexible scheduling',
    'contact_required': 'Contact us for availability and custom options'
}

for urgency, count in df_rag['urgency_level'].value_counts().items():
    trigger = urgency_triggers.get(urgency, 'Standard booking process')
    print(f"   • {urgency.title()} ({count} products): '{trigger}'")

print(f"\n3. 💰 PRICE-BASED LEAD QUALIFICATION:")
price_strategies = {
    'budget_friendly': 'Easy booking, quick quotes, emphasize value',
    'mid_range': 'Detailed consultation, comparison with alternatives',
    'premium': 'Personal consultation, custom planning, VIP treatment',
    'contact_for_pricing': 'Lead capture required, custom quote process'
}

for tier, count in df_rag['price_tier'].value_counts().items():
    strategy = price_strategies.get(tier, 'Standard process')
    percentage = (count / total_products) * 100
    print(f"   • {tier.replace('_', ' ').title()} ({count} products, {percentage:.1f}%): {strategy}")

print(f"\n4. 🎁 CONVERSION BOOSTERS:")
if special_offers > 0:
    offer_percentage = (special_offers / total_products) * 100
    print(f"   • {special_offers} products ({offer_percentage:.1f}%) have special offers")
    print(f"   • Use time-limited language: 'Special price ends soon!'")
    print(f"   • Add savings calculator: 'You save X MAD with this offer!'")
else:
    print(f"   • No special offers detected - consider adding promotional pricing")

print(f"\n5. 📝 LEAD CAPTURE POINTS:")
print(f"   • After product interest expression (primary capture)")
print(f"   • Before pricing information (premium products)")
print(f"   • After urgency messaging (limited availability)")
print(f"   • During comparison requests (multiple products)")

print(f"\n6. 🤖 BOT PERSONALITY RECOMMENDATIONS:")
print(f"   • Tone: Professional yet friendly (hospitality industry)")
print(f"   • Language: Bilingual support (French/Arabic for Morocco)")
print(f"   • Expertise: Food service and event planning specialist")
print(f"   • Approach: Consultative selling, not pushy")

print(f"\n7. 📊 SUCCESS METRICS TO TRACK:")
print(f"   • Lead capture rate by product category")
print(f"   • Conversion rate by price tier")
print(f"   • Response time to urgency messaging")
print(f"   • Quote request completion rate")
print(f"   • Customer satisfaction with bot interactions")

# Generate sample conversation starters based on data
print(f"\n8. 💬 SAMPLE CONVERSATION STARTERS BY CATEGORY:")
categories = df_rag['Categories'].str.split(',').explode().str.strip().value_counts().head(5)

for category, count in categories.items():
    if 'buffet' in category.lower():
        starter = f"Looking for a buffet? I can help you find the perfect spread for your event!"
    elif 'mariage' in category.lower() or 'wedding' in category.lower():
        starter = f"Planning a wedding? Let me show you our beautiful wedding catering options!"
    elif 'soutenance' in category.lower():
        starter = f"Celebrating your graduation? We have special packages for academic celebrations!"
    else:
        starter = f"Interested in {category.lower()}? I'd love to help you explore our options!"
    
    print(f"   • {category} ({count} products): '{starter}'")

print(f"\n✅ Configuration complete! Your lead generation bot is optimized for:")
print(f"   🎯 Maximum lead capture")
print(f"   💰 Revenue optimization") 
print(f"   ⚡ Urgency-driven conversions")
print(f"   🎁 Promotional effectiveness")
print(f"   📱 Mobile-friendly interactions")